In [1]:
!pip install transformers datasets torch scikit-learn accelerate peft

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    XLMRobertaTokenizer,
    XLMRobertaForSequenceClassification,
    TrainingArguments,
    Trainer
)
import torch
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [3]:
from tqdm import tqdm

In [4]:
df = pd.read_csv("/content/train.csv")
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['label']
)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [5]:
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

Map:   0%|          | 0/95724 [00:00<?, ? examples/s]

Map:   0%|          | 0/23932 [00:00<?, ? examples/s]

In [6]:
num_labels = 15  # As mentioned
model = XLMRobertaForSequenceClassification.from_pretrained(
    "xlm-roberta-base",
    num_labels=num_labels
)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "value"],  # ✅ Correct for XLM-R
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)


In [8]:
model = get_peft_model(model, lora_config)

In [9]:
# Verify
def print_trainable_params(model):
    trainable = 0
    total = 0
    for _, param in model.named_parameters():
        total += param.numel()
        if param.requires_grad:
            trainable += param.numel()
    print(f"Trainable params: {trainable} ({100 * trainable/total:.4f}% of total {total})")

print_trainable_params(model)

Trainable params: 1191951 (0.4268% of total 279247134)


In [10]:
# ============================================
# 📏 Step 7: Define metrics
# ============================================
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}


In [11]:
# ============================================
# ⚡ Step 8: GPU check
# ============================================
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n🚀 Using device: {device}")




🚀 Using device: cuda


In [12]:
# ============================================
# 🏋️ Step 9: Training Arguments
# ============================================
training_args = TrainingArguments(
    output_dir="./xlmroberta_peft",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    num_train_epochs=5,
    learning_rate=2e-4,          # LoRA often benefits from a slightly higher lr
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=torch.cuda.is_available(),
)

In [13]:
# ============================================
# 🧑‍🏫 Step 10: Initialize Trainer
# ============================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-308073751.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
# ====================================================
# Train
# ====================================================
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 21f3002511 (21f3002511-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.585400,1.514937,0.516798,0.495553,0.510965,0.516798
2,1.411700,1.398026,0.554112,0.525875,0.542970,0.554112
3,1.259200,1.312308,0.586662,0.569679,0.578024,0.586662
4,1.211200,1.265110,0.599699,0.584271,0.592637,0.599699
5,1.279200,1.242362,0.606761,0.594825,0.595317,0.606761


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/m

TrainOutput(global_step=59830, training_loss=1.4255440506830714, metrics={'train_runtime': 5501.7481, 'train_samples_per_second': 86.994, 'train_steps_per_second': 10.875, 'total_flos': 6.384873113450496e+16, 'train_loss': 1.4255440506830714, 'epoch': 5.0})

In [15]:
# ====================================================
# Evaluate
# ====================================================
print("Evaluation:")
print(trainer.evaluate())

Evaluation:


{'eval_loss': 1.2423615455627441, 'eval_accuracy': 0.6067608223299348, 'eval_f1': 0.5948254249519304, 'eval_precision': 0.59531650676426, 'eval_recall': 0.6067608223299348, 'eval_runtime': 97.8035, 'eval_samples_per_second': 244.695, 'eval_steps_per_second': 30.592, 'epoch': 5.0}


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
# To load later:
# from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer
# from peft import PeftModel
# base = XLMRobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)
# peft_model = PeftModel.from_pretrained(base, peft_save_dir)
# tokenizer = XLMRobertaTokenizer.from_pretrained(peft_save_dir)


In [18]:
# ====================================================
# Quick predict example
# ====================================================
# example = "𐤀𐤁𐤂 example ancient sentence"
# inputs = tokenizer(example, return_tensors="pt", truncation=True, padding=True, max_length=256)
# inputs = {k: v.to(device) for k, v in inputs.items()}
# model.to(device)
# model.eval()
# with torch.no_grad():
#     outputs = model(**inputs)
#     pred = torch.argmax(outputs.logits, dim=-1).item()
# print("Predicted label:", pred)

In [19]:
# ====================================================
# 🚀 Step 11: Load the best model and predict
# ====================================================
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer
from peft import PeftModel
import pandas as pd
import torch
from datasets import Dataset
import os

# Define the path to the saved PEFT model checkpoint
peft_save_dir = "./xlmroberta_peft/checkpoint-59830" # Adjust if your checkpoint name is different

# Load the base model
MODEL_NAME = "xlm-roberta-base"
num_labels = 15 # Ensure this matches your training setup
base_model = XLMRobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

# Load the PEFT model
model = PeftModel.from_pretrained(base_model, peft_save_dir)

# Load the tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained(peft_save_dir)

# Load the test data
test_data_path = "/content/test.csv"  # Assuming your test data is here
test_df = pd.read_csv(test_data_path)

# Create a Hugging Face Dataset from the test DataFrame
test_dataset = Dataset.from_pandas(test_df)

# Tokenize the test dataset
# Use the existing tokenize_function
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set the format to PyTorch tensors, keeping the 'id' column
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'id'])


# Set the model to evaluation mode and move to device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# Prepare for predictions
predictions = []
ids = []

# Predict in batches to manage memory
batch_size = 8 # Adjust batch size if needed
for i in tqdm(range(0, len(test_dataset), batch_size), desc="Predicting"):
    batch = test_dataset[i : i + batch_size]
    inputs = {k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask']}
    batch_ids = batch['id']

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1).cpu().numpy() # Move predictions to CPU

    predictions.extend(preds)
    ids.extend(batch_ids)


# Create submission DataFrame
submission_df = pd.DataFrame({'id': ids, 'label': predictions})

# Save submission file
submission_df.to_csv("submission.csv", index=False)

print("Submission file created successfully: submission.csv")

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/29914 [00:00<?, ? examples/s]

Predicting: 100%|██████████| 3740/3740 [06:32<00:00,  9.53it/s]


Submission file created successfully: submission.csv
